In [243]:
# Importamos librerías de análisis de datos
%matplotlib inline
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('mode.chained_assignment', None) # Deshabilita SettingWithCopyWarning. Ojo.

In [244]:
# Importamos utilidades y modelos de sklearn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [245]:
df = pd.read_csv('../input/prices/train.csv', parse_dates=['fecha'])
df_test = pd.read_csv('../input/prices/test.csv')
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,nan,2.00,1.00,2.00,80.00,80.00,23533.00,nan,nan,2015-08-23,0.00,0.00,0.00,0.00,0.00,2273000.00
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.00,3.00,2.00,2.00,268.00,180.00,24514.00,19.31,-99.23,2013-06-28,0.00,0.00,0.00,1.00,1.00,3600000.00
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.00,3.00,2.00,2.00,144.00,166.00,48551.00,nan,nan,2015-10-17,0.00,0.00,0.00,0.00,0.00,1200000.00
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.00,2.00,1.00,1.00,63.00,67.00,53666.00,19.30,-99.69,2012-03-09,0.00,0.00,0.00,1.00,1.00,650000.00
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.00,2.00,1.00,1.00,95.00,95.00,47835.00,nan,nan,2016-06-07,0.00,0.00,0.00,0.00,0.00,1150000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.00,2.00,2.00,1.00,67.00,nan,53666.00,nan,nan,2015-02-08,0.00,0.00,0.00,0.00,0.00,650000.00
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.00,3.00,3.00,3.00,200.00,250.00,51954.00,19.29,-99.69,2014-07-10,0.00,0.00,0.00,1.00,1.00,1940000.00
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.00,2.00,1.00,2.00,138.00,138.00,50003995.00,nan,nan,2015-03-03,0.00,0.00,0.00,0.00,0.00,3400000.00
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.00,4.00,0.00,4.00,235.00,137.00,24162.00,19.37,-99.08,2014-12-26,1.00,0.00,0.00,1.00,1.00,2890000.00


In [246]:
def asignarMetros(metroscubiertos, metrostotales):
    if (metroscubiertos != 0 and metrostotales == 0 or metroscubiertos> metrostotales):
        return metroscubiertos
    else:
        return metrostotales

In [247]:
def agregarAnioMesYPrecioDelDolarALaFecha(dfReceived):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    return dfReceived

In [248]:
import pandas as pd
import numpy as np
import seaborn as sns

dolar= pd.read_csv('../input/prices/Datos histricos USD_MXN.csv')
dolar.columns = ["fecha", "last", "opening", "max", "min", "std"]
dolar.drop(["std", "opening", "opening", "max", "min"], axis=1 ,inplace=True)
#Paso a formato fecha correctamente y luego creo columnas mes y año
dolar['date'] = pd.to_datetime(dolar['fecha'], format="%b %Y")
dolar.drop(["fecha"], axis=1, inplace=True)
dolar.columns= ['price', "date"]
#El csv ponia los numeros con punto en vez de coma.
dolar['price'] = dolar['price'].replace(',', '.', regex=True).astype(float)
dolar["year"] = dolar.date.dt.year
dolar['month'] = dolar.date.dt.month
dolar.drop(["date"], axis=1, inplace=True)

In [249]:
def setDataFrameWithDollarPrice(dfReceived, dolar):
    dfReceived['fecha'] = pd.to_datetime(dfReceived['fecha'])
    dfReceived['year'] = dfReceived['fecha'].dt.year
    dfReceived['month']= dfReceived['fecha'].dt.month

    #Agrego la columna precio en dolar por propiedad, en el dataframe recibido.
    dfReceived= pd.merge(dfReceived, dolar, on=['year', 'month'], how='left')
    #Filtro dejando las cosas útiles.
    #Chequeado de no dejar alguna importante afuera
    dfReceived = dfReceived.filter(["id", "descripcion", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha',
                   'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                   'centroscomercialescercanos', 'price', 'precio'])
    print(dfReceived.columns)


    
    #Limpiando las propiedades tipo casa, no pueden no tener al menos una habitacion o baño y no pueden
    #tener metros no cubiertos
    filtro1 = dfReceived["tipodepropiedad"].isin(["Apartamento", "Casa", "Casa en condominio",
                                        "Casa uso de suelo", "Rancho", "Quinta Vacacional"])
    dfCasas = dfReceived[filtro1]
    dfCasas = dfCasas[np.isfinite(dfCasas['habitaciones'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['banos'])]
    dfCasas = dfCasas[np.isfinite(dfCasas['metroscubiertos'])]
    dfCasas.garages.fillna(0, inplace=True)

    dfReceived = pd.merge(dfCasas, dfReceived, on=["id", "descripcion", "tipodepropiedad", "habitaciones",'garages', 'banos', 'ciudad',
                   'provincia','metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'year', 'month',
                   'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad',
                  'centroscomercialescercanos', 'price', 'precio'], how='inner')

    dfReceived.rename(columns={"price": "precioDelDolar"}, inplace = True)
    return dfReceived

In [250]:
df_train = setDataFrameWithDollarPrice(df, dolar)
df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)

Index(['id', 'descripcion', 'tipodepropiedad', 'habitaciones', 'garages',
       'banos', 'ciudad', 'provincia', 'metroscubiertos', 'metrostotales',
       'idzona', 'fecha', 'year', 'month', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'antiguedad',
       'centroscomercialescercanos', 'price', 'precio'],
      dtype='object')


In [251]:
def addColumns(dfReceived):
    dfReceived['metrostotales'].fillna(0, inplace=True)
    dfReceived['metrostotales'] = dfReceived.apply(lambda x: asignarMetros(x['metroscubiertos'], x['metrostotales']),axis=1)
    dfReceived["cant_ambientes"] = dfReceived["habitaciones"] + dfReceived["garages"] + dfReceived["banos"]
    dfReceived["metros_no_cubiertos"] = dfReceived["metrostotales"] - dfReceived["metroscubiertos"]

In [252]:
addColumns(df_train)
addColumns(df_test)

df_train

,id,descripcion,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,fecha,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,cant_ambientes,metros_no_cubiertos
0,254099,"depto. interior de 80.15m2, consta de sala com...",Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,80.00,80.00,23533.00,2015-08-23,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,2273000.00,5.00,0.00
1,53461,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,3.00,2.00,2.00,La Magdalena Contreras,Distrito Federal,268.00,268.00,24514.00,2013-06-28,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,3600000.00,7.00,0.00
2,247984,descripcion \nla mejor ubicacion residencial e...,Casa,3.00,2.00,2.00,Tonalá,Jalisco,144.00,166.00,48551.00,2015-10-17,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,1200000.00,7.00,22.00
3,209067,casa en privada con caseta de vigilancia casas...,Casa,2.00,1.00,1.00,Zinacantepec,Edo. de México,63.00,67.00,53666.00,2012-03-09,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,650000.00,4.00,4.00
4,185997,bonito departamento en excelentes condiciones ...,Apartamento,2.00,1.00,1.00,Zapopan,Jalisco,95.00,95.00,47835.00,2016-06-07,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,1150000.00,4.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174992,119879,vendo casa en bosques de ica residencial a 10 ...,Casa,2.00,2.00,1.00,Zinacantepec,Edo. de México,67.00,67.00,53666.00,2015-02-08,2015,2,0.00,0.00,0.00,0.00,0.00,0.00,14.95,650000.00,5.00,0.00
174993,259178,"casa con un jardin amplio, un cuarto de servic...",Casa,3.00,3.00,3.00,Toluca,Edo. de México,200.00,250.00,51954.00,2014-07-10,2014,7,0.00,0.00,0.00,1.00,0.00,1.00,13.22,1940000.00,9.00,50.00
174994,131932,"departamento con excelente ubicación, muy cerc...",Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,138.00,138.00,50003995.00,2015-03-03,2015,3,0.00,0.00,0.00,0.00,20.00,0.00,15.26,3400000.00,5.00,0.00
174995,146867,"casa sola, dividida en cuatro departamentos de...",Casa,4.00,0.00,4.00,Iztapalapa,Distrito Federal,235.00,235.00,24162.00,2014-12-26,2014,12,1.00,0.00,0.00,1.00,20.00,1.00,14.75,2890000.00,8.00,0.00


In [253]:
def addFeature(listaPalabras, nombreDelFeature):
    df_train[nombreDelFeature] = 0
    df_test[nombreDelFeature] = 0
    for palabra in listaPalabras:
        df_train.loc[df_train['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
        df_test.loc[df_test['descripcion'].str.contains(palabra)==True, nombreDelFeature] = 1
    print("Feature (train) " + nombreDelFeature + ":" + str(df_train[nombreDelFeature].sum()))
    print("Feature (test) " + nombreDelFeature + ":" + str(df_test[nombreDelFeature].sum()))

In [254]:
addFeature(["luminoso"], "es_luminoso")
addFeature(["amplio"], "es_amplio")
addFeature(["balcon"], "tiene_balcon")
addFeature(["nuevo"], "es_nuevo")
addFeature(["buen estado", "excelente estado", "excelentes condiciones"], "buen_estado")
addFeature(["excelente ubicacion", "buena ubicaion", "ubicacion"], "excelente_ubicacion")
addFeature(["privada", "barrio privado", "privado"], "es_privado")

Feature (train) es_luminoso:323
Feature (test) es_luminoso:94
Feature (train) es_amplio:26909
Feature (test) es_amplio:9325
Feature (train) tiene_balcon:3816
Feature (test) tiene_balcon:1232
Feature (train) es_nuevo:8067
Feature (test) es_nuevo:2434
Feature (train) buen_estado:7938
Feature (test) buen_estado:2545
Feature (train) excelente_ubicacion:6334
Feature (test) excelente_ubicacion:2111
Feature (train) es_privado:21899
Feature (test) es_privado:7273


In [255]:
# Droppeamos strings y columnas complejas y repetidas
drop_cols_train = ['descripcion', 'fecha']
drop_cols_test = ['titulo', 'descripcion', 'direccion', 'lat', 'lng', 'fecha']
df_train = df_train.drop(drop_cols_train, axis=1).copy()

df_test = agregarAnioMesYPrecioDelDolarALaFecha(df_test)
df_test = df_test.drop(drop_cols_test, axis=1).copy()
print(f"Columnas ({len(df_train.columns)}): {df_train.columns.tolist()}")
df_train.head()

Columnas (29): ['id', 'tipodepropiedad', 'habitaciones', 'garages', 'banos', 'ciudad', 'provincia', 'metroscubiertos', 'metrostotales', 'idzona', 'year', 'month', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'antiguedad', 'centroscomercialescercanos', 'precioDelDolar', 'precio', 'cant_ambientes', 'metros_no_cubiertos', 'es_luminoso', 'es_amplio', 'tiene_balcon', 'es_nuevo', 'buen_estado', 'excelente_ubicacion', 'es_privado']


,id,tipodepropiedad,habitaciones,garages,banos,ciudad,provincia,metroscubiertos,metrostotales,idzona,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,cant_ambientes,metros_no_cubiertos,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado
0,254099,Apartamento,2.00,1.00,2.00,Benito Juárez,Distrito Federal,80.00,80.00,23533.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,2273000.00,5.00,0.00,0,0,0,0,0,0,0
1,53461,Casa en condominio,3.00,2.00,2.00,La Magdalena Contreras,Distrito Federal,268.00,268.00,24514.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,3600000.00,7.00,0.00,0,0,0,0,1,0,0
2,247984,Casa,3.00,2.00,2.00,Tonalá,Jalisco,144.00,166.00,48551.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,1200000.00,7.00,22.00,0,1,0,0,0,1,1
3,209067,Casa,2.00,1.00,1.00,Zinacantepec,Edo. de México,63.00,67.00,53666.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,650000.00,4.00,4.00,0,0,0,0,0,0,1
4,185997,Apartamento,2.00,1.00,1.00,Zapopan,Jalisco,95.00,95.00,47835.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,1150000.00,4.00,0.00,0,0,0,0,1,1,0


In [256]:
df_test.dtypes

id                              int64
tipodepropiedad                object
ciudad                         object
provincia                      object
antiguedad                    float64
habitaciones                  float64
garages                       float64
banos                         float64
metroscubiertos               float64
metrostotales                 float64
idzona                        float64
gimnasio                      float64
usosmultiples                 float64
piscina                       float64
escuelascercanas              float64
centroscomercialescercanos    float64
year                            int64
month                           int64
price_x                       float64
cant_ambientes                float64
metros_no_cubiertos           float64
es_luminoso                     int64
es_amplio                       int64
tiene_balcon                    int64
es_nuevo                        int64
buen_estado                     int64
excelente_ub

In [257]:
display(df_train.isnull().sum())
numeric_columns_with_nulls = list(set(df_train.columns[df_train.isnull().sum() > 0].tolist()) - 
                                  set(['tipodepropiedad', 'ciudad', 'provincia']))
print(numeric_columns_with_nulls)

id                                0
tipodepropiedad                   0
habitaciones                      0
garages                           0
banos                             0
ciudad                          178
provincia                        48
metroscubiertos                   0
metrostotales                     0
idzona                        18320
year                              0
month                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
antiguedad                    24006
centroscomercialescercanos        0
precioDelDolar                    0
precio                            0
cant_ambientes                    0
metros_no_cubiertos               0
es_luminoso                       0
es_amplio                         0
tiene_balcon                      0
es_nuevo                          0
buen_estado                       0
excelente_ubicacion         

['idzona', 'antiguedad']


In [258]:
ciudades_en_train = df_train.ciudad.unique()
ciudades_en_test = df_test.ciudad.unique()

tipodepropiedad_en_train = df_train.tipodepropiedad.unique()
tipodepropiedad_en_test = df_test.tipodepropiedad.unique()

provincia_en_train = df_train.provincia.unique()
provincia_en_test = df_test.provincia.unique()

In [259]:
# Para los nulls numéricos, usar un Imputer con strategy mean (reemplazamos los NaN por el promedio)
# Para no leakear, spliteamos el dataset antes
df_train_ohe = pd.get_dummies(df_train, dummy_na=True)
print("First")
df_test_ohe = pd.get_dummies(df_test, drop_first=True)

First


In [260]:
X = df_train_ohe.drop("precio", axis=1)
y = df_train_ohe['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)


display(df_train_ohe)

,id,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,cant_ambientes,metros_no_cubiertos,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,tipodepropiedad_Apartamento,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_nan,ciudad_Abasolo,ciudad_Acajete,ciudad_Acambay,ciudad_Acapulco de Juárez,ciudad_Acateno,ciudad_Acatzingo,ciudad_Acayucan,ciudad_Acolman,ciudad_Actopan,ciudad_Acuña,ciudad_Agua Dulce,ciudad_Agua Prieta,ciudad_Aguascalientes,ciudad_Ahome,ciudad_Ahuacatlán,ciudad_Ahualulco de Mercado,ciudad_Alamo Temapache,...,ciudad_Zapotlán de Juárez,ciudad_Zapotlán el Grande,ciudad_Zaragoza,ciudad_Zempoala,ciudad_Zihuatanejo de Azueta,ciudad_Zimatlán de Alvarez,ciudad_Zinacantepec,ciudad_ZirAndaro,ciudad_Zitácuaro,ciudad_Zumpahuacán,ciudad_Zumpango,ciudad_otra,ciudad_ácatlán,ciudad_ácaxochitlán,ciudad_ácámbaro,ciudad_áutlán de Navarro,ciudad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,2.00,1.00,2.00,80.00,80.00,23533.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,2273000.00,5.00,0.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,3.00,2.00,2.00,268.00,268.00,24514.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,3600000.00,7.00,0.00,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,3.00,2.00,2.00,144.00,166.00,48551.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,1200000.00,7.00,22.00,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,2.00,1.00,1.00,63.00,67.00,53666.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,650000.00,4.00,4.00,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,2.00,1.00,1.00,95.00,95.00,47835.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,1150000.00,4.00,0.00,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174992,119879,2.00,2.00,1.00,67.00,67.00,53666.00,2015,2,0.00,0.00,0.00,0.00,0.00,0.00,14.95,650000.00,5.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174993,259178,3.00,3.00,3.00,200.00,250.00,51954.00,2014,7,0.00,0.00,0.00,1.00,0.00,1.00,13.22,1940000.00,9.00,50.00,0,1,0,0

In [261]:
# Con dummy_na=True, creamos la categoria "Es nulo" como una coordenada más de los one-hot vectors
# Comentar: ¿Hay leaks acá? ¿Sí / No? ¿Por qué?
#df2 = pd.get_dummies(df2, dummy_na=True)
print(f"Cantidad de columnas después del one-hot-encoding: {len(df_train_ohe.columns)}")
df_train_ohe.head(10)

Cantidad de columnas después del one-hot-encoding: 742


,id,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,year,month,gimnasio,usosmultiples,piscina,escuelascercanas,antiguedad,centroscomercialescercanos,precioDelDolar,precio,cant_ambientes,metros_no_cubiertos,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,tipodepropiedad_Apartamento,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_nan,ciudad_Abasolo,ciudad_Acajete,ciudad_Acambay,ciudad_Acapulco de Juárez,ciudad_Acateno,ciudad_Acatzingo,ciudad_Acayucan,ciudad_Acolman,ciudad_Actopan,ciudad_Acuña,ciudad_Agua Dulce,ciudad_Agua Prieta,ciudad_Aguascalientes,ciudad_Ahome,ciudad_Ahuacatlán,ciudad_Ahualulco de Mercado,ciudad_Alamo Temapache,...,ciudad_Zapotlán de Juárez,ciudad_Zapotlán el Grande,ciudad_Zaragoza,ciudad_Zempoala,ciudad_Zihuatanejo de Azueta,ciudad_Zimatlán de Alvarez,ciudad_Zinacantepec,ciudad_ZirAndaro,ciudad_Zitácuaro,ciudad_Zumpahuacán,ciudad_Zumpango,ciudad_otra,ciudad_ácatlán,ciudad_ácaxochitlán,ciudad_ácámbaro,ciudad_áutlán de Navarro,ciudad_nan,provincia_Aguascalientes,provincia_Baja California Norte,provincia_Baja California Sur,provincia_Campeche,provincia_Chiapas,provincia_Chihuahua,provincia_Coahuila,provincia_Colima,provincia_Distrito Federal,provincia_Durango,provincia_Edo. de México,provincia_Guanajuato,provincia_Guerrero,provincia_Hidalgo,provincia_Jalisco,provincia_Michoacán,provincia_Morelos,provincia_Nayarit,provincia_Nuevo León,provincia_Oaxaca,provincia_Puebla,provincia_Querétaro,provincia_Quintana Roo,provincia_San luis Potosí,provincia_Sinaloa,provincia_Sonora,provincia_Tabasco,provincia_Tamaulipas,provincia_Tlaxcala,provincia_Veracruz,provincia_Yucatán,provincia_Zacatecas,provincia_nan
0,254099,2.00,1.00,2.00,80.00,80.00,23533.00,2015,8,0.00,0.00,0.00,0.00,nan,0.00,16.75,2273000.00,5.00,0.00,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,53461,3.00,2.00,2.00,268.00,268.00,24514.00,2013,6,0.00,0.00,0.00,1.00,10.00,1.00,12.95,3600000.00,7.00,0.00,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,247984,3.00,2.00,2.00,144.00,166.00,48551.00,2015,10,0.00,0.00,0.00,0.00,5.00,0.00,16.50,1200000.00,7.00,22.00,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,209067,2.00,1.00,1.00,63.00,67.00,53666.00,2012,3,0.00,0.00,0.00,1.00,1.00,1.00,12.81,650000.00,4.00,4.00,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,185997,2.00,1.00,1.00,95.00,95.00,47835.00,2016,6,0.00,0.00,0.00,0.00,10.00,0.00,18.28,1150000.00,4.00,0.00,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,126147,2.00,1.00,1.00,75.00,90.00,23650.00,2014,3,0.00,0.00,0.00,0.00,5.00,1.00,13.06,1100000.00,4.00,15.00,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,139233,3.00,1.00,2.00,140.00,160.00,73510.00,2016,2,0.00,0.00,0.00,0.00,nan,0.00,18.14,1150000.00,6.00,20.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5013,4.00,2.00,3.00,293.00,293.00,130510.00,2016,10,0.00,0.00,0.00,0.00,2.00,0.00,18.86,4200000.00,9.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [262]:
numeric_columns_with_nulls = list(set(df_train_ohe.columns[df_train_ohe.isnull().sum() > 0].tolist()) \
                                  - set(['tipodepropiedad', 'provincia', 'ciudad']))
print(numeric_columns_with_nulls)

['idzona', 'antiguedad']


In [263]:
from sklearn.impute import SimpleImputer
for c in numeric_columns_with_nulls:
    imp = SimpleImputer()
    X_train[c] = imp.fit_transform(X_train[[c]])
    X_test[c] = imp.transform(X_test[[c]])
    df_test_ohe[c] = imp.transform(df_test_ohe[[c]])
    
print ("Finished")

Finished


In [264]:
from catboost import CatBoostRegressor


cbr = CatBoostRegressor(iterations= 1000,
                    learning_rate = 0.06,
                    depth=13,
                    eval_metric='MAE',
                    bagging_temperature = 0.2,
                    metric_period = 75,
                    od_wait=100)

In [265]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators = 1000, 
     learning_rate=0.11, max_depth=10,
     max_features=None, max_leaf_nodes=None,
     min_samples_leaf=3,
     min_samples_split=2, min_weight_fraction_leaf=0.0)


In [266]:
import xgboost as xgbt
xgb = xgbt.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.04,
                max_depth = 9, min_child_weight = 1, n_estimators = 700, nthread = -1, subsample = 0.7, silent = 1)

In [267]:
from mlxtend.regressor import StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(gbr, cbr, xgb),
                                meta_regressor= xgb,
                                use_features_in_secondary=True)

NameError: name 'xgboost' is not defined

In [ ]:
print("Fitting.")
cbr.fit(X_train, y_train,
    eval_set=(X_test,y_test),
    #cat_features=categorical_features_pos,
    use_best_model=True,
    verbose=True)

print("Catboost regressor: " + str(model.score(X_test, y_test)))

gbr.fit(X_train, y_train)
print("Gradient boosting regressor: " + str(gbr.score(X_test, y_test)))

xgb.fit(X_train, y_train)
print("Gradient boosting regressor: " + str(xgb.score(X_test, y_test)))

In [208]:
for ciudad in ciudades_en_train:
    if ciudad not in ciudades_en_test:
        df_test_ohe['ciudad_'+ str(ciudad)] = 0
        
for provincia in provincia_en_train:
    if provincia not in provincia_en_test:
        df_test_ohe['provincia_'+ str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        df_test_ohe['tipodepropiedad_'+ str(tipodepropiedad)] = 0

In [217]:
some_list = ['tipodepropiedad_Apartamento', 'ciudad_Abasolo', 'provincia_Aguascalientes', 'tipodepropiedad_nan']
for item in some_list:
    df_test_ohe[item] = 0

In [219]:
df_test_ohe.drop("price_y", axis = "columns",inplace = True)
df_test_ohe.rename(columns={"price_x": "precioDelDolar"}, inplace = True)
df_test_ohe

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,year,month,precioDelDolar,cant_ambientes,metros_no_cubiertos,es_luminoso,es_amplio,tiene_balcon,es_nuevo,buen_estado,excelente_ubicacion,es_privado,tipodepropiedad_Bodega comercial,tipodepropiedad_Casa,tipodepropiedad_Casa en condominio,tipodepropiedad_Casa uso de suelo,tipodepropiedad_Departamento Compartido,tipodepropiedad_Duplex,tipodepropiedad_Edificio,tipodepropiedad_Huerta,tipodepropiedad_Inmuebles productivos urbanos,tipodepropiedad_Local Comercial,tipodepropiedad_Local en centro comercial,tipodepropiedad_Lote,tipodepropiedad_Nave industrial,tipodepropiedad_Oficina comercial,tipodepropiedad_Otros,tipodepropiedad_Quinta Vacacional,tipodepropiedad_Rancho,tipodepropiedad_Terreno,tipodepropiedad_Terreno comercial,tipodepropiedad_Terreno industrial,tipodepropiedad_Villa,ciudad_Abejones,ciudad_Acajete,ciudad_Acambay,ciudad_Acaponeta,...,ciudad_Contepec,ciudad_El Fuerte,ciudad_Aquismón,ciudad_Cananea,ciudad_San Pedro Mixtepec -Dto. 26 -,ciudad_Jocotitlán,ciudad_Angostura,ciudad_Tototlán,ciudad_Reyes Etla,ciudad_Dr. Belisario Domínguez,ciudad_Jamapa,ciudad_Yuriria,ciudad_Xaltocan,ciudad_Nadadores,ciudad_Tezontepec de Aldama,ciudad_Chiconcuac,ciudad_Urique,ciudad_Tamiahua,ciudad_Río Grande,ciudad_Teapa,ciudad_San Mateo Río Hondo,ciudad_Nautla,ciudad_Tejupilco,ciudad_Ciudad Fernández,ciudad_Tamazunchale,ciudad_San Juan Guelavía,ciudad_Calvillo,ciudad_Batopilas,ciudad_Tepalcatepec,ciudad_San Pablo Huitzo,ciudad_Santa María del Oro,ciudad_Nanchital de Lázaro Cárdenas del Río,ciudad_Tlaquiltenango,ciudad_Altotonga,ciudad_Pinal de Amoles,ciudad_Marín,ciudad_Sabinas Hidalgo,ciudad_Tepeaca,ciudad_ZirAndaro,ciudad_Salvatierra,ciudad_Ocosingo,ciudad_Gral. Bravo,ciudad_Bocoyna,ciudad_Omealca,ciudad_Pánuco de Coronado,provincia_nan,tipodepropiedad_Apartamento,ciudad_Abasolo,provincia_Aguascalientes,tipodepropiedad_nan
0,4941,29.00,3.00,nan,4.00,300.00,300.00,2590591.03,0.00,0.00,0.00,0.00,0.00,2013,7,12.73,nan,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,51775,7.97,1.00,1.00,1.00,67.00,67.00,113851.00,0.00,0.00,0.00,0.00,0.00,2015,10,16.50,3.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,115253,0.00,2.00,1.00,2.00,87.00,100.00,23620.00,0.00,0.00,0.00,0.00,1.00,2015,5,15.38,5.00,13.00,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,299321,2.00,2.00,2.00,2.00,86.00,86.00,129347.00,0.00,0.00,0.00,0.00,0.00,2015,4,15.35,6.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,173570,10.00,2.00,1.00,1.00,80.00,80.00,57125.00,0.00,0.00,0.00,1.00,1.00,2013,8,13.38,4.00,0.00,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,20.00,4.00,3.00,3.00,291.00,291.00,275741.00,0.00,0.00,0.00,1.00,1.00,2015,9,16.92,10.00,0.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [220]:
df_test_ohe = df_test_ohe[list(X_train.columns.values)]

In [221]:
y_test

167055   1055000.00
9813     1520000.00
125958   2490000.00
94896    3750000.00
124802    900000.00
            ...    
132461   1798000.00
28085    2645000.00
124743   9393000.00
18966    1300000.00
67003    1550000.00
Name: precio, Length: 17500, dtype: float64

In [222]:
y_false_pred = model.predict(X_test)
print(y_false_pred)

[1454420.93423676 1818359.08953981 2522727.47801254 ... 9287034.76617705
 1504141.19992263 1557389.39054363]


In [223]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_false_pred)

475712.1227438588

In [224]:
#Cosas para entregar
X_real_test = df_test_ohe
y_real_pred = model.predict(X_real_test)
y_real_pred

array([4924085.6012147 ,  843961.62180589, 2016182.47880536, ...,
        852423.39861092, 1433110.18555765, 2493012.34292303])

In [225]:
res = pd.DataFrame(y_real_pred, index= df_test_ohe.index, columns=['precio'])
res2 = res
res2['id'] = df_test_ohe["id"]
res = res.reset_index(drop=True)
res.set_index('id', inplace = True)

In [226]:
res

,precio
id,
4941,4924085.60
51775,843961.62
115253,2016182.48
299321,1029330.16
173570,591318.70
...,...
75094,4480256.51
171847,825392.19
138313,852423.40


In [233]:
res.to_csv("catBoostWithNewFeatures.csv", header=True)